In [ ]:
import os
import json
from typing import Iterable
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
tickers = ['AAPL', 'GOOGL', 'NVDA']

In [ ]:
def get_stocks_prices_form_db(
        symbols: Iterable[str], db_file_path: str
        ) -> dict[str, dict[str, dict[str, float | None]]]:
    connection = sqlite3.connect(db_file_path)
    cursor = connection.cursor()
    # print(cur.execute('PRAGMA table_list').fetchall())
    stocks_prices = {}
    for symbol in symbols:
        try:
            row = cursor.execute(
                f"SELECT date, open, high, low, close FROM '{symbol.lower()}' ORDER BY date"
                ).fetchall()
            stocks_prices[symbol.upper()] = (
                {i[0]: {'Open': i[1], 'High': i[2], 'Low': i[3], 'Close': i[4]} for i in row}
                )
        except sqlite3.OperationalError:
            continue
    connection.close()

    return stocks_prices

In [ ]:
prices = get_stocks_prices_form_db(tickers, '../files_repo/prices.db')

In [ ]:
close_prices: dict[str, dict[str, float | None]] = {}
for ticker in tickers:
    close_prices[ticker] = {k: v['Close'] for k, v in prices[ticker].items()}

In [ ]:
fig = plt.figure(figsize=(15, 18), tight_layout=True)
for n, ticker in enumerate(tickers):
    plot_prices = pd.Series(close_prices[ticker]).sort_index().iloc[-500:]
    plot_prices.index = pd.DatetimeIndex(plot_prices.index)
    
    plt.subplot(6, 5, n+1)
    plot_prices.plot(lw=1)
    plot_prices.rolling(20).mean().plot(lw=1)
    plot_prices.rolling(60).mean().plot(lw=1)
    plot_prices.rolling(180).mean().plot(lw=1)
    plt.legend(['close', 'sma(20)', 'sma(60)', 'sma(180)'], fontsize=6)
    plt.title(f'{ticker}')
    plt.xticks(fontsize=6)
    plt.yticks(fontsize=6)
    plt.grid(visible=True)
    